In [1]:
from itertools import dropwhile, chain
from operator import attrgetter

from geoip2.database import Reader
from ipyleaflet import Map, Marker, Polygon, basemaps

from src import traceroute, unique_justseen, my_ip, iter_deque

In [2]:
reader = Reader('./data/GeoLite2-City.mmdb')

In [3]:
def geo_trace(host):
    _ = better_trace(host)
    _ = map(geoip, _)
    _ = unique_justseen(_)
    return _


def better_trace(host):
    _ = traceroute(host)
    _ = dropwhile(lambda s: s.startswith('192.168'), _)
    _ = chain([my_ip()], _)
    _ = unique_justseen(_)
    return _


def geoip(ip):
    l = reader.city(ip).location
    return (l.latitude, l.longitude)

In [4]:
m = Map(zoom=1, basemap=basemaps.CartoDB.DarkMatter)
display(m)

first = True
for a, b in iter_deque(geo_trace('cnn.co.jp'), 2):
    if first:
        m += Marker(location=a, draggable=False)
        first = False
    m += Marker(location=b, draggable=False)
    m += Polygon(locations=[a, b])

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': …